In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

## Loss Functions

In this assignment, we will learn about loss functions. We will use a create a neural network and measure the model's performance using different loss functions.

In [2]:
import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

<IPython.core.display.Javascript object>

In [3]:
housing = pd.read_csv("./housing.csv")

<IPython.core.display.Javascript object>

In [4]:
housing.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


<IPython.core.display.Javascript object>

We will use the dataset above to predict housing prices using various features about each house. Our first step is to check for missing data. 

In [5]:
# Answer below:
housing.isna().mean()


Id               0.000000
MSSubClass       0.000000
MSZoning         0.000000
LotFrontage      0.177397
LotArea          0.000000
Street           0.000000
Alley            0.937671
LotShape         0.000000
LandContour      0.000000
Utilities        0.000000
LotConfig        0.000000
LandSlope        0.000000
Neighborhood     0.000000
Condition1       0.000000
Condition2       0.000000
BldgType         0.000000
HouseStyle       0.000000
OverallQual      0.000000
OverallCond      0.000000
YearBuilt        0.000000
YearRemodAdd     0.000000
RoofStyle        0.000000
RoofMatl         0.000000
Exterior1st      0.000000
Exterior2nd      0.000000
MasVnrType       0.005479
MasVnrArea       0.005479
ExterQual        0.000000
ExterCond        0.000000
Foundation       0.000000
BsmtQual         0.025342
BsmtCond         0.025342
BsmtExposure     0.026027
BsmtFinType1     0.025342
BsmtFinSF1       0.000000
BsmtFinType2     0.026027
BsmtFinSF2       0.000000
BsmtUnfSF        0.000000
TotalBsmtSF 

<IPython.core.display.Javascript object>

Remove columns that contain more than 30% of missing data. After removing those columns, remove the rows that contain at least one observation that is missing.

In [6]:
# Answer below:
housing = housing.drop(
    columns=["Alley", "FireplaceQu", "PoolQC", "Fence", "MiscFeature"]
)
housing.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,0,12,2008,WD,Normal,250000


<IPython.core.display.Javascript object>

In [7]:
housing = housing.dropna()
housing.isna().mean()

Id               0.0
MSSubClass       0.0
MSZoning         0.0
LotFrontage      0.0
LotArea          0.0
Street           0.0
LotShape         0.0
LandContour      0.0
Utilities        0.0
LotConfig        0.0
LandSlope        0.0
Neighborhood     0.0
Condition1       0.0
Condition2       0.0
BldgType         0.0
HouseStyle       0.0
OverallQual      0.0
OverallCond      0.0
YearBuilt        0.0
YearRemodAdd     0.0
RoofStyle        0.0
RoofMatl         0.0
Exterior1st      0.0
Exterior2nd      0.0
MasVnrType       0.0
MasVnrArea       0.0
ExterQual        0.0
ExterCond        0.0
Foundation       0.0
BsmtQual         0.0
BsmtCond         0.0
BsmtExposure     0.0
BsmtFinType1     0.0
BsmtFinSF1       0.0
BsmtFinType2     0.0
BsmtFinSF2       0.0
BsmtUnfSF        0.0
TotalBsmtSF      0.0
Heating          0.0
HeatingQC        0.0
CentralAir       0.0
Electrical       0.0
1stFlrSF         0.0
2ndFlrSF         0.0
LowQualFinSF     0.0
GrLivArea        0.0
BsmtFullBath     0.0
BsmtHalfBath 

<IPython.core.display.Javascript object>

There are some categorical variables that contain numeric data and some that do not. Print the type of each column to first see whether there is an issue with misclassification of column type.

In [8]:
# Answer below:
housing.dtypes


Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
LotShape          object
LandContour       object
Utilities         object
LotConfig         object
LandSlope         object
Neighborhood      object
Condition1        object
Condition2        object
BldgType          object
HouseStyle        object
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
RoofStyle         object
RoofMatl          object
Exterior1st       object
Exterior2nd       object
MasVnrType        object
MasVnrArea       float64
ExterQual         object
ExterCond         object
Foundation        object
BsmtQual          object
BsmtCond          object
BsmtExposure      object
BsmtFinType1      object
BsmtFinSF1         int64
BsmtFinType2      object
BsmtFinSF2         int64
BsmtUnfSF          int64
TotalBsmtSF        int64
Heating           object
HeatingQC         object


<IPython.core.display.Javascript object>

We see that month sold and year sold are not variables that describe a feature of the house. While they do have relevance if we create a model containing a time series element, we will not include them here. Drop these columns. Also, remove the id column

In [9]:
# Answer below
housing = housing.drop(columns=["MoSold", "YrSold", "Id"])

<IPython.core.display.Javascript object>

In [10]:
housing.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,0,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,0,WD,Normal,250000


<IPython.core.display.Javascript object>

Using the information about the column types, identify all the variables that will be converted into dummy variables. Include at least one numeric variable that you think should be converted as well.

In [11]:
# Answer below:
cat_cols = ['MSZoning','LotFrontage','Street','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1','Condition2','BldgType','HouseStyle','OverallQual','OverallCond','YearBuilt','YearRemodAdd','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','ExterQual','ExterCond','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','BsmtFinSF2','Heating','HeatingQC','CentralAir','Electrical','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','KitchenQual','TotRmsAbvGrd','Functional','Fireplaces','GarageType','GarageFinish','GarageCars','GarageQual','GarageCond','PavedDrive','SaleType','SaleCondition']


<IPython.core.display.Javascript object>

Convert the columns you selected above into dummy variables

In [12]:
# Answer below:
housing = pd.get_dummies(housing, columns=cat_cols,drop_first=True)


<IPython.core.display.Javascript object>

Split the data into train and test with 20% of data in test.

In [13]:
# Answer below
X = housing.drop(columns=["SalePrice"])
y = np.array(housing["SalePrice"])
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=13
)

<IPython.core.display.Javascript object>

Create a model with 5 layers. The first layer should be a dense layer that takes in the input, the last layer should be of size 1. You determine the remaining layer sizes.

Use a linear activation for the output layer.

In [14]:
# Answer below
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

<IPython.core.display.Javascript object>

In [15]:
model = Sequential()
model.add(Dense(512, input_dim=X_train_scaled.shape[1], activation="relu"))
model.add(Dense(256, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(1, activation="linear"))

<IPython.core.display.Javascript object>

Compile the model with the RMSprop optimizer and mean square error loss. Use the MSE as a metric. Set batch size to 100 and epochs to 200. Fit the model and report the results.

In [16]:
# Answer below:
model.compile(loss="mse", optimizer="RMSprop", metrics=["mse"])

<IPython.core.display.Javascript object>

In [17]:
model.fit(
    X_train_scaled,
    y_train,
    validation_data=(X_test_scaled, y_test),
    epochs=200,
    batch_size=100,
)

Train on 875 samples, validate on 219 samples
Epoch 1/200
875/875 [==============================] - 1s 750us/sample - loss: 42591289929.1429 - mse: 42591293440.0000 - val_loss: 38911021766.7215 - val_mse: 38911021056.0000
Epoch 2/200
875/875 [==============================] - 0s 82us/sample - loss: 42191095983.5429 - mse: 42191097856.0000 - val_loss: 37976095327.8539 - val_mse: 37976096768.0000
Epoch 3/200
875/875 [==============================] - 0s 77us/sample - loss: 40492622643.2000 - mse: 40492621824.0000 - val_loss: 35293234966.2100 - val_mse: 35293233152.0000
Epoch 4/200
875/875 [==============================] - 0s 77us/sample - loss: 36666809314.7429 - mse: 36666810368.0000 - val_loss: 30280706066.7032 - val_mse: 30280706048.0000
Epoch 5/200
875/875 [==============================] - 0s 68us/sample - loss: 30219602856.2286 - mse: 30219603968.0000 - val_loss: 22890843458.6301 - val_mse: 22890842112.0000
Epoch 6/200
875/875 [==============================] - 0s 73us/sample - l

875/875 [==============================] - 0s 72us/sample - loss: 474726218.0571 - mse: 474726240.0000 - val_loss: 845954990.7580 - val_mse: 845955008.0000
Epoch 92/200
875/875 [==============================] - 0s 72us/sample - loss: 472768300.8000 - mse: 472768288.0000 - val_loss: 864369133.5890 - val_mse: 864369088.0000
Epoch 93/200
875/875 [==============================] - 0s 68us/sample - loss: 480236609.8286 - mse: 480236640.0000 - val_loss: 863097564.0548 - val_mse: 863097536.0000
Epoch 94/200
875/875 [==============================] - 0s 66us/sample - loss: 451511094.8571 - mse: 451511104.0000 - val_loss: 856216190.2466 - val_mse: 856216192.0000
Epoch 95/200
875/875 [==============================] - 0s 71us/sample - loss: 481104707.6571 - mse: 481104704.0000 - val_loss: 867166576.2192 - val_mse: 867166592.0000
Epoch 96/200
875/875 [==============================] - 0s 68us/sample - loss: 464925234.2857 - mse: 464925184.0000 - val_loss: 845765862.8676 - val_mse: 845765888.0000

Epoch 187/200
875/875 [==============================] - 0s 75us/sample - loss: 217461033.1429 - mse: 217461056.0000 - val_loss: 1029386029.5890 - val_mse: 1029386048.0000
Epoch 188/200
875/875 [==============================] - 0s 71us/sample - loss: 217996827.8857 - mse: 217996816.0000 - val_loss: 1060500309.9178 - val_mse: 1060500288.0000
Epoch 189/200
875/875 [==============================] - 0s 72us/sample - loss: 223774666.5143 - mse: 223774672.0000 - val_loss: 1041156650.0822 - val_mse: 1041156672.0000
Epoch 190/200
875/875 [==============================] - 0s 67us/sample - loss: 198273069.2571 - mse: 198273088.0000 - val_loss: 1050259664.0731 - val_mse: 1050259648.0000
Epoch 191/200
875/875 [==============================] - 0s 66us/sample - loss: 229623301.9429 - mse: 229623312.0000 - val_loss: 1039093799.7443 - val_mse: 1039093760.0000
Epoch 192/200
875/875 [==============================] - 0s 73us/sample - loss: 203992043.4286 - mse: 203992032.0000 - val_loss: 1065113351.

<IPython.core.display.Javascript object>

Next, do the same but with mean absolute error loss. Use both MSE and MAE as metrics. Compare the results.

In [20]:
# Answer below:
model2 = Sequential()
model2.add(Dense(512, input_dim=X_train_scaled.shape[1], activation="relu"))
model2.add(Dense(256, activation="relu"))
model2.add(Dense(128, activation="relu"))
model2.add(Dense(64, activation="relu"))
model2.add(Dense(1, activation="linear"))

<IPython.core.display.Javascript object>

In [21]:
model2.compile(loss="mae", optimizer="RMSprop", metrics=["mse", "mae"])

<IPython.core.display.Javascript object>

In [22]:
model2.fit(
    X_train_scaled,
    y_train,
    validation_data=(X_test_scaled, y_test),
    epochs=200,
    batch_size=100,
)

Train on 875 samples, validate on 219 samples
Epoch 1/200
875/875 [==============================] - 1s 703us/sample - loss: 188542.5112 - mse: 42581872640.0000 - mae: 188542.5156 - val_loss: 180262.3484 - val_mse: 38858735616.0000 - val_mae: 180262.3594
Epoch 2/200
875/875 [==============================] - 0s 85us/sample - loss: 187015.3268 - mse: 42001891328.0000 - mae: 187015.3125 - val_loss: 176633.0177 - val_mse: 37552435200.0000 - val_mae: 176633.0156
Epoch 3/200
875/875 [==============================] - 0s 89us/sample - loss: 180774.1286 - mse: 39712477184.0000 - mae: 180774.1250 - val_loss: 166292.5723 - val_mse: 33977759744.0000 - val_mae: 166292.5625
Epoch 4/200
875/875 [==============================] - 0s 85us/sample - loss: 166164.0004 - mse: 34592456704.0000 - mae: 166164.0000 - val_loss: 145347.7326 - val_mse: 27387709440.0000 - val_mae: 145347.7188
Epoch 5/200
875/875 [==============================] - 0s 81us/sample - loss: 139000.2699 - mse: 26321711104.0000 - mae: 

Epoch 78/200
875/875 [==============================] - 0s 68us/sample - loss: 14098.2811 - mse: 810312896.0000 - mae: 14098.2812 - val_loss: 17581.1487 - val_mse: 859413120.0000 - val_mae: 17581.1484
Epoch 79/200
875/875 [==============================] - 0s 71us/sample - loss: 14238.8325 - mse: 803769472.0000 - mae: 14238.8330 - val_loss: 17477.1251 - val_mse: 913517248.0000 - val_mae: 17477.1250
Epoch 80/200
875/875 [==============================] - 0s 78us/sample - loss: 13573.5634 - mse: 799806656.0000 - mae: 13573.5635 - val_loss: 17276.3610 - val_mse: 890610304.0000 - val_mae: 17276.3613
Epoch 81/200
875/875 [==============================] - 0s 70us/sample - loss: 14179.2593 - mse: 798993920.0000 - mae: 14179.2598 - val_loss: 17204.4303 - val_mse: 855395840.0000 - val_mae: 17204.4297
Epoch 82/200
875/875 [==============================] - 0s 74us/sample - loss: 13471.8588 - mse: 774375616.0000 - mae: 13471.8584 - val_loss: 17224.4565 - val_mse: 839026496.0000 - val_mae: 17224.

Epoch 119/200
875/875 [==============================] - 0s 79us/sample - loss: 12701.1157 - mse: 725227008.0000 - mae: 12701.1152 - val_loss: 17470.2808 - val_mse: 777718848.0000 - val_mae: 17470.2793
Epoch 120/200
875/875 [==============================] - 0s 78us/sample - loss: 12700.0033 - mse: 721407872.0000 - mae: 12700.0039 - val_loss: 17878.7971 - val_mse: 742955904.0000 - val_mae: 17878.7969
Epoch 121/200
875/875 [==============================] - 0s 79us/sample - loss: 12608.5158 - mse: 718553280.0000 - mae: 12608.5156 - val_loss: 17176.1903 - val_mse: 725497984.0000 - val_mae: 17176.1895
Epoch 122/200
875/875 [==============================] - 0s 70us/sample - loss: 12705.2408 - mse: 726313216.0000 - mae: 12705.2402 - val_loss: 17443.7143 - val_mse: 770566848.0000 - val_mae: 17443.7148
Epoch 123/200
875/875 [==============================] - 0s 72us/sample - loss: 12476.1069 - mse: 725136512.0000 - mae: 12476.1064 - val_loss: 17553.3121 - val_mse: 721670400.0000 - val_mae: 1

Epoch 200/200
875/875 [==============================] - 0s 81us/sample - loss: 11597.8366 - mse: 658221184.0000 - mae: 11597.8369 - val_loss: 17375.1048 - val_mse: 638317120.0000 - val_mae: 17375.1055


<IPython.core.display.Javascript object>

Finally, try your model using mean squared logarithmic error. Compare the results.

In [23]:
# Answer below:
model3 = Sequential()
model3.add(Dense(512, input_dim=X_train_scaled.shape[1], activation="relu"))
model3.add(Dense(256, activation="relu"))
model3.add(Dense(128, activation="relu"))
model3.add(Dense(64, activation="relu"))
model3.add(Dense(1, activation="linear"))


<IPython.core.display.Javascript object>

In [24]:
model3.compile(loss="msle", optimizer="RMSprop", metrics=["mse", "mae"])

<IPython.core.display.Javascript object>

In [25]:
model3.fit(
    X_train_scaled,
    y_train,
    validation_data=(X_test_scaled, y_test),
    epochs=200,
    batch_size=100,
)

Train on 875 samples, validate on 219 samples
Epoch 1/200
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
875/875 [==============================] - 1s 741us/sample - loss: 84.4976 - mse: 42596450304.0000 - mae: 188581.2500 - val_loss: 51.5010 - val_mse: 38972391424.0000 - val_mae: 180574.6406
Epoch 2/200
875/875 [==============================] - 0s 85us/sample - loss: 43.0015 - mse: 42507800576.0000 - mae: 188346.3906 - val_loss: 33.6744 - val_mse: 38832959488.0000 - val_mae: 180191.4844
Epoch 3/200
875/875 [==============================] - 0s 92us/sample - loss: 29.5692 - mse: 42310217728.0000 - mae: 187822.3438 - val_loss: 24.1467 - val_mse: 38565593088.0000 - val_mae: 179454.4531
Epoch 4/200
875/875 [==============================] - 0s 81us/sample - loss: 21.5345 - mse: 41951592448.0000 - mae: 186873.7656 - val_loss: 17.7724 - val_mse: 38108221440.0000 - val_mae: 178186.9375
Epoch 5/200
875/875 [==============================] - 0s 8

Epoch 37/200
875/875 [==============================] - 0s 79us/sample - loss: 0.0182 - mse: 1273350144.0000 - mae: 19681.5801 - val_loss: 0.0228 - val_mse: 1499070464.0000 - val_mae: 21271.4023
Epoch 38/200
875/875 [==============================] - 0s 66us/sample - loss: 0.0179 - mse: 1235643264.0000 - mae: 19424.6953 - val_loss: 0.0229 - val_mse: 1535515008.0000 - val_mae: 20502.2715
Epoch 39/200
875/875 [==============================] - 0s 67us/sample - loss: 0.0167 - mse: 1166567296.0000 - mae: 18684.6074 - val_loss: 0.0221 - val_mse: 1453641088.0000 - val_mae: 20041.8926
Epoch 40/200
875/875 [==============================] - 0s 68us/sample - loss: 0.0173 - mse: 1139557376.0000 - mae: 19092.1738 - val_loss: 0.0211 - val_mse: 1339594112.0000 - val_mae: 20127.3457
Epoch 41/200
875/875 [==============================] - 0s 71us/sample - loss: 0.0170 - mse: 1120987008.0000 - mae: 18855.4277 - val_loss: 0.0204 - val_mse: 1326150016.0000 - val_mae: 19409.3242
Epoch 42/200
875/875 [===

875/875 [==============================] - 0s 68us/sample - loss: 0.0062 - mse: 463572192.0000 - mae: 10880.5479 - val_loss: 0.0177 - val_mse: 780606272.0000 - val_mae: 18605.1973
Epoch 122/200
875/875 [==============================] - 0s 67us/sample - loss: 0.0062 - mse: 448292032.0000 - mae: 11129.5303 - val_loss: 0.0177 - val_mse: 783364096.0000 - val_mae: 18352.5977
Epoch 123/200
875/875 [==============================] - 0s 72us/sample - loss: 0.0070 - mse: 486286944.0000 - mae: 11601.3623 - val_loss: 0.0188 - val_mse: 821869248.0000 - val_mae: 18875.7324
Epoch 124/200
875/875 [==============================] - 0s 67us/sample - loss: 0.0055 - mse: 428915360.0000 - mae: 10257.2744 - val_loss: 0.0176 - val_mse: 787740672.0000 - val_mae: 18396.8516
Epoch 125/200
875/875 [==============================] - 0s 70us/sample - loss: 0.0068 - mse: 471822848.0000 - mae: 11880.3672 - val_loss: 0.0172 - val_mse: 771220736.0000 - val_mae: 18219.3867
Epoch 126/200
875/875 [=====================

<IPython.core.display.Javascript object>